# 암환자 유전체 데이터 기반 암종 분류 AI 모델 개발


# library loading

In [ ]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
!git clone https://github.com/jeongminia/Cancer-Classification.git

Cloning into 'Cancer-Classification'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 184 (delta 80), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (184/184), 12.82 MiB | 6.44 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [ ]:
!unzip -qq '/content/Cancer-Classification/data/open.zip'

# data loading

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
print(train.head(5))
print(train.tail(5))

           ID SUBCLASS    A2M AAAS AADAT AARS1 ABAT ABCA1 ABCA2 ABCA3  ...  \
0  TRAIN_0000    KIPAN     WT   WT    WT    WT   WT    WT    WT    WT  ...   
1  TRAIN_0001     SARC     WT   WT    WT    WT   WT    WT    WT    WT  ...   
2  TRAIN_0002     SKCM  R895R   WT    WT    WT   WT    WT    WT    WT  ...   
3  TRAIN_0003     KIRC     WT   WT    WT    WT   WT    WT    WT    WT  ...   
4  TRAIN_0004   GBMLGG     WT   WT    WT    WT   WT    WT    WT    WT  ...   

  ZNF292 ZNF365 ZNF639 ZNF707 ZNFX1 ZNRF4 ZPBP ZW10 ZWINT ZYX  
0     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
1     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
2     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
3     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
4     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  

[5 rows x 4386 columns]
              ID SUBCLASS A2M AAAS AADAT AARS1 ABAT ABCA1 ABCA2 ABCA3  ...  \
6196  TRAIN_6196     LUAD  WT   WT    WT    

In [ ]:
print(test.head(5))
print(test.tail(5))

          ID A2M AAAS AADAT AARS1 ABAT  ABCA1 ABCA2 ABCA3 ABCA4  ... ZNF292  \
0  TEST_0000  WT   WT    WT    WT   WT     WT    WT    WT    WT  ...     WT   
1  TEST_0001  WT   WT    WT    WT   WT  R587Q    WT    WT    WT  ...     WT   
2  TEST_0002  WT   WT    WT    WT   WT     WT    WT    WT    WT  ...     WT   
3  TEST_0003  WT   WT    WT    WT   WT     WT    WT    WT    WT  ...     WT   
4  TEST_0004  WT   WT    WT    WT   WT     WT    WT    WT    WT  ...     WT   

  ZNF365 ZNF639 ZNF707 ZNFX1    ZNRF4 ZPBP ZW10 ZWINT ZYX  
0     WT     WT     WT    WT       WT   WT   WT    WT  WT  
1     WT     WT     WT    WT  I383Sfs   WT   WT    WT  WT  
2     WT     WT     WT    WT       WT   WT   WT    WT  WT  
3     WT     WT     WT    WT       WT   WT   WT    WT  WT  
4     WT     WT     WT    WT       WT   WT   WT    WT  WT  

[5 rows x 4385 columns]
             ID A2M AAAS AADAT AARS1 ABAT  ABCA1        ABCA2 ABCA3   ABCA4  \
2541  TEST_2541  WT   WT    WT    WT   WT     WT           WT

In [ ]:
len(train), len(test)

(6201, 2546)

# preprocessing

1. 결측값 처리
2. 아미노산 변이 치환
  - WT(정상)
  - missense ← 알파벳숫자**알파벳** (치환)
  - insertion ← **-**숫자알파벳 or 알파벳숫자**ins** (삽입)
  - deletion ← 알파벳숫자**del** (삭제)
  - delins ← 알파벳숫자**delins**알파벳 (집단치환)
  - frameshift ← 알파벳숫자**fs**
  - nonsense ← 알파벳숫자 * (조기종결)

  2.1. category1
  
  2.2. category2

  2.3. category3

3. column WT 차원축소
4. 파생변수 생성 mutation_cnt
5. Custom Numeric Encoding
6. Subclass_encoding

### 결측값 처리

In [ ]:
# NaN 개수와 위치 확인
train_nan = train.isna().sum().sum()
train_nan_columns = train.columns[train.isna().any()].tolist()
print(f"train NaN 개수: {train_nan}")
print("NaN 값이 있는 열:")
print(train_nan_columns)

test_nan = test.isna().sum().sum()
test_nan_columns = test.columns[test.isna().any()].tolist()
print(f"test NaN 개수: {test_nan}")
print("NaN 값이 있는 열:")
print(test_nan_columns)

train NaN 개수: 0
NaN 값이 있는 열:
[]
test NaN 개수: 237
NaN 값이 있는 열:
['AK2', 'ATP6V1H', 'CCRL2', 'CFP', 'CNOT2', 'CRAT', 'DPYSL4', 'GUK1', 'IER3', 'INHBB', 'KCNH1', 'MYL1', 'NDUFV1', 'NUDT4', 'POLD2', 'PTCH1', 'PTGES3', 'RBM5', 'SCAMP1', 'SCNN1A', 'SLC25A28', 'SYBU', 'TARS1', 'TMEM97', 'TNFAIP6']


In [ ]:
# NaN 값을 'WT'로 치환
train.fillna('WT', inplace=True)
test.fillna('WT', inplace=True)

train_nan = train.isna().sum().sum()
print(f"처리 후 train NaN 개수: {train_nan}")

test_nan = test.isna().sum().sum()
print(f"처리 후 test NaN 개수: {test_nan}")

처리 후 train NaN 개수: 0
처리 후 test NaN 개수: 0


### 아미노산 변이 치환

In [ ]:
def process_value(value):
    # 0. WT 그대로 유지
    if value == 'WT':
        return 0

    # 1. 띄어쓰기 기준으로 변이 파트 분리
    parts = value.split()
    processed_sum = 0

    for part in parts:
        # 1.1 WT: 0
        if part == 'WT':
            continue

        # 1.2 알파벳 + 숫자 + 알파벳
        elif re.match(r'^([A-Za-z*-]*)\d+([A-Za-z*-]*)$', part):
            match = re.match(r'^([A-Za-z*-]*)\d+([A-Za-z*-]*)$', part)
            prefix = match.group(1)   # 숫자 앞의 문자 그룹
            suffix = match.group(2)   # 숫자 뒤의 문자 그룹

            # 1.2.1 숫자 앞뒤의 문자 그룹이 같으면 WT -> 0
            if prefix.isupper() and suffix.isupper() and prefix == suffix:
                continue

            # 1.2.2 앞뒤 문자 그룹이 다르면 missense -> 10**0
            elif prefix.isupper() and suffix.isupper() and prefix != suffix:
                processed_sum += 10**0

            # 1.2.3 뒤의 문자 그룹에 '*'이 들어가면 nonsense -> 10**3
            elif '*' in suffix:
                processed_sum += 10**3

            # 1.2.4 뒤의 문자 그룹에 'fs'가 들어가면 frameshift -> 10**2
            elif 'fs' in suffix:
                processed_sum += 10**2

            # 1.2.5 뒤에 문자 그룹에 'delins'이 들어가면 delins-> (10**1) * (변이 아미노산의 개수)
            elif 'delins' in suffix:
                processed_sum += (len(prefix)+len(suffix))*(10**1)

            # 1.2.6 뒤에 문자 그룹에 'del'이 들어가면 deletion -> 10**1
            elif 'del' in suffix or not(suffix):
                processed_sum += 10**1

            # 1.2.7 예외사항 출력
            else:
                print(part)

        # 1.3 숫자_숫자알파벳>알파벳
        elif re.match(r'^(\d+)_(\d+)([A-Za-z*-]+>[A-Za-z*-]+|del)$', part):
            match = re.match(r'^(\d+)_(\d+)([A-Za-z*-]+>[A-Za-z*-]+|del)$', part)
            prenum = int(match.group(1))  # 앞 위치
            postnum = int(match.group(2))  # 뒷 위치
            variant = match.group(3)  # 변이 정보 그룹

            # 1.3.1 >가 있는 경우
            if '>' in variant:
                before, after = variant.split('>')  # '>' 기준으로 나눔
                # 1.3.1.1 문자 그룹이 같은 경우 -> WT -> 0
                if before.isupper() and after.isupper() and before == after:
                    continue

                # 1.3.1.2 *가 있는 경우 -> nonsense -> 10**3
                elif '*' in after:
                    processed_sum += 10**3

                # 1.3.1.3 문자 그룹이 다른 경우 -> missense -> (2)*(10**0)
                elif before.isupper() and after.isupper() and before != after:
                    processed_sum += (2)*(10**0)

                else:
                    print(part)

            # 1.3.2 > 대신 'del'이 있는 경우 -> deletion -> (10**1) * (변이 아미노산의 개수)
            elif 'del' in variant:
                processed_sum += (postnum-prenum+1)*(10**1)

            # 1.3.3 예외사항 출력
            else :
                print(part)


        # 1.4 알파벳숫자_알파벳숫자변이정보
        elif re.match(r'^([A-Za-z]+)(\d+)_([A-Za-z]+)?(\d+)([A-Za-z]+)$', part):
            match = re.match(r'^([A-Za-z]+)(\d+)_([A-Za-z]+)?(\d+)([A-Za-z]+)$', part)
            prenum = int(match.group(2))   # 앞 위치
            postnum = int(match.group(4))  # 뒷 위치
            variant = match.group(5)  # 변이정보

            # 1.4.1 'delins'가 있는 경우 -> delins -> (10**1) * (변이 아미노산의 개수)
            if 'del' in variant and 'ins' in variant:
                processed_sum += ((postnum-prenum+1)+(len(variant)-3))*(10**1)

            # 1.4.2 'del'만 있는 경우 -> deletion -> (10**1) * (변이 아미노산의 개수)
            elif 'del' in variant and not('ins' in variant):
                processed_sum += (postnum-prenum+1)*(10**1)

            # 1.4.3 'ins'만 있는 경우 -> insertion -> (10**1) * (변이 아미노산의 개수)
            elif 'ins' in variant and not('del' in variant):
                processed_sum += (len(variant)-3)*(10**1)

            # 1.4.4 예외사항 출력
            else :
                print(part)

    return processed_sum

In [ ]:
# train 데이터프레임 전처리
for col in train.columns[2:]:  # ID와 SUBCLASS 제외
    train[col] = train[col].apply(process_value)

# 최종 데이터 확인
print(train)

              ID SUBCLASS  A2M  AAAS  AADAT  AARS1  ABAT  ABCA1  ABCA2  ABCA3  \
0     TRAIN_0000    KIPAN    0     0      0      0     0      0      0      0   
1     TRAIN_0001     SARC    0     0      0      0     0      0      0      0   
2     TRAIN_0002     SKCM    0     0      0      0     0      0      0      0   
3     TRAIN_0003     KIRC    0     0      0      0     0      0      0      0   
4     TRAIN_0004   GBMLGG    0     0      0      0     0      0      0      0   
...          ...      ...  ...   ...    ...    ...   ...    ...    ...    ...   
6196  TRAIN_6196     LUAD    0     0      0      0     0      0      0      0   
6197  TRAIN_6197      LGG    0     0      0      0     0      0      0      0   
6198  TRAIN_6198     COAD    0     0      0      0     0      0      0      0   
6199  TRAIN_6199     TGCT    0     0      0      0     0      0      0      0   
6200  TRAIN_6200     SKCM    0     0      0      0     0      0      0      0   

      ...  ZNF292  ZNF365  

In [ ]:
# test 데이터프레임 전처리
for col in test.columns[1:]:  # ID 제외
    test[col] = test[col].apply(process_value)

# 최종 데이터 확인
print(test)

             ID  A2M  AAAS  AADAT  AARS1  ABAT  ABCA1  ABCA2  ABCA3  ABCA4  \
0     TEST_0000    0     0      0      0     0      0      0      0      0   
1     TEST_0001    0     0      0      0     0      1      0      0      0   
2     TEST_0002    0     0      0      0     0      0      0      0      0   
3     TEST_0003    0     0      0      0     0      0      0      0      0   
4     TEST_0004    0     0      0      0     0      0      0      0      0   
...         ...  ...   ...    ...    ...   ...    ...    ...    ...    ...   
2541  TEST_2541    0     0      0      0     0      0      0      0      0   
2542  TEST_2542    0     0      0      0     0      0      0      0      0   
2543  TEST_2543    0     0      0      0     0      1      0      1      1   
2544  TEST_2544    0     0      0      0     0      0      0      0      1   
2545  TEST_2545    0     0      0      0     0      0      0      0      0   

      ...  ZNF292  ZNF365  ZNF639  ZNF707  ZNFX1  ZNRF4  ZPBP  

### column WT 차원축소
- WT로만 구성된 컬럼 삭제

In [ ]:
train_wt_columns = train.columns[(train == 0).all()].tolist()
test_wt_columns = test.columns[(test == 0).all()].tolist()

print('데이터셋에서 값이 모두 WT인 열 개수')
print('trainSet: ', len(train_wt_columns))
print('testSet: ', len(test_wt_columns))

데이터셋에서 값이 모두 WT인 열 개수
trainSet:  159
testSet:  29


In [ ]:
cnt = 0
both_wt_columns = []
for gene in train_wt_columns:
  if gene in test_wt_columns:
    both_wt_columns.append(gene)
    cnt += 1

print('두 데이터셋에서 값이 모두 WT인 열 개수:', cnt)
both_wt_columns

두 데이터셋에서 값이 모두 WT인 열 개수: 21


['BOLA2',
 'CROCCP2',
 'EEIG1',
 'G6PC1',
 'GPX4',
 'H2AC25',
 'HBBP1',
 'HYCC2',
 'MIX23',
 'MYL11',
 'NHERF1',
 'NHERF4',
 'PALS1',
 'PHB1',
 'PTTG3P',
 'PVT1',
 'RIGI',
 'SELENOP',
 'SELENOW',
 'SKIC3',
 'XIST']

In [ ]:
train_df = train.drop(columns=both_wt_columns)
test_df = test.drop(columns=both_wt_columns)

# CSV out

In [ ]:
train_df.to_csv('preprocessed_trainv2.csv', index=False)
test_df.to_csv('preprocessed_testv2.csv', index=False)

# Plus Code

## Subclass encoding
: LabelEncoder

In [ ]:
# SUBCLASS 가 범주형이기 때문에 LabelEncoder 사용
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

원래 레이블: ACC, 변환된 숫자: 0
원래 레이블: BLCA, 변환된 숫자: 1
원래 레이블: BRCA, 변환된 숫자: 2
원래 레이블: CESC, 변환된 숫자: 3
원래 레이블: COAD, 변환된 숫자: 4
원래 레이블: DLBC, 변환된 숫자: 5
원래 레이블: GBMLGG, 변환된 숫자: 6
원래 레이블: HNSC, 변환된 숫자: 7
원래 레이블: KIPAN, 변환된 숫자: 8
원래 레이블: KIRC, 변환된 숫자: 9
원래 레이블: LAML, 변환된 숫자: 10
원래 레이블: LGG, 변환된 숫자: 11
원래 레이블: LIHC, 변환된 숫자: 12
원래 레이블: LUAD, 변환된 숫자: 13
원래 레이블: LUSC, 변환된 숫자: 14
원래 레이블: OV, 변환된 숫자: 15
원래 레이블: PAAD, 변환된 숫자: 16
원래 레이블: PCPG, 변환된 숫자: 17
원래 레이블: PRAD, 변환된 숫자: 18
원래 레이블: SARC, 변환된 숫자: 19
원래 레이블: SKCM, 변환된 숫자: 20
원래 레이블: STES, 변환된 숫자: 21
원래 레이블: TGCT, 변환된 숫자: 22
원래 레이블: THCA, 변환된 숫자: 23
원래 레이블: THYM, 변환된 숫자: 24
원래 레이블: UCEC, 변환된 숫자: 25
